# <font color='darkred'>Tese de Mestrado</font>
## Modelling

In [ ]:
# Importação de Bibliotecas
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.utils import class_weight
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
import lightgbm as lgb
import logging

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_excel('df_final.xlsx')
df.head()

In [ ]:
df.shape

In [ ]:
df.columns

#### <font color='darkred'>Tratamento</font>
#### Identificação dos Valores Omissose Substiuição po NA

In [ ]:
total_omissos = df.isna().sum().sum()
print("\nNúmero total de valores omissos no DataFrame:", total_omissos)

In [ ]:
# Substituição dos valores omissos por NA
df.replace('', np.nan, inplace=True)

#### 11 Anos no Privado

In [ ]:
df_filtrado = df.copy()

In [ ]:
df_filtrado = df_filtrado[~df_filtrado.apply(lambda row: row.astype(str).str.contains('Não Trabalhou').any(), axis=1)]
df_filtrado.shape

In [ ]:
total_omissos = df_filtrado.isna().sum().sum()
print("\nNúmero total de valores omissos no DataFrame:", total_omissos)

- A percentagem de pessoas que estão há 11 anos no privado é aproximadamente 29,73% da amostra em análise. 

#### Categorias "Ignorada"
É sabido que nas variáveis tipo_contr, habil e prof_3d existe a categoria "Ignorada". Tendo em consideração que se tratam de variáveis ordinais e que serão utilizadas desta forma no Modelo de Regressão Logística, averiguar-se-á se estas categorias efetivamente existem na amostra selecionada. 
- Variável tipo_contr

In [ ]:
tipo_contr1_columns = [col for col in df_filtrado.columns if col.startswith('tipo_contr1')]
unique_values_set = set()

for col in tipo_contr1_columns:
    unique_values_set.update(df_filtrado[col].unique())

contains_8 = 8 in unique_values_set
print(f"O valor 8 (Ignorada) está presente nas colunas 'tipo_contr1': {contains_8}")

- Variável habil

In [ ]:
habil_columns = [col for col in df_filtrado.columns if col.startswith('habil1')]
unique_values_set = set()

for col in habil_columns:
    unique_values_set.update(df_filtrado[col].unique())

contains_9 = 9 in unique_values_set
print(f"O valor 9 (Ignorada) está presente nas colunas 'habil1': {contains_9}")

- Variável prof_3d

In [ ]:
prof_3d_columns = [col for col in df_filtrado.columns if col.startswith('prof_3d_')]
unique_values_set = set()

for col in prof_3d_columns:
    unique_values_set.update(df_filtrado[col].unique())

contains_9999 = 9999 in unique_values_set
print(f"O valor 9999 (Ignorada) está presente nas colunas 'prof_3d': {contains_9999}")

Tendo em consideração que todas as variáveis têm esta categoria, proceder-se-á à eliminação destes trabalhadores, por forma a não enviesar os resultados.
- Eliminação dos Trabalhadores que contém esta categoria

In [ ]:
# Nº de "Ignorada" em cada coluna
ignorado_valores = {
    'habil1': [9],
    'tipo_contr1': [8],
    'prof_3d': [9999]
}

for var, valores in ignorado_valores.items():
    for ano in range(9, 20):
        col = f'{var}_{ano:02d}'
        if col in df_filtrado.columns:
            ignorados = df_filtrado[col].isin(valores).sum()
            print(f'{col} - Valores Ignorados ({valores}): {ignorados}')

In [ ]:
df_trab_filtrada = df_filtrado.copy()

for var, valores in ignorado_valores.items():
    for ano in range(9, 20):
        col = f'{var}_{ano:02d}'
        if col in df_trab_filtrada.columns:
            df_trab_filtrada = df_trab_filtrada[~df_trab_filtrada[col].isin(valores)]

print(f"Dimensões do DataFrame após remoção: {df_trab_filtrada.shape}")

Após este tratamento, foi eliminada 6,49% da amostra considerada para este modelo, tendo no final, 50 778 trabalhadores.

#### Correção de Na's

In [ ]:
valores_ausentes = df_trab_filtrada.isna().sum()
colunas_com_na = valores_ausentes[valores_ausentes > 0]
colunas_com_na

- Das colunas de interesse para o Modelo, apenas a variável rganho tem valores omissos. Tendo em consideração que são apenas 4, será feita a substituição deste pelo último valor rganho com informação.

In [ ]:
# Preencher valores ausentes no 'rganho' com o valor do ano anterior
for ano in range(10, 20):
    col_atual = f'rganho_h_{ano:02d}'
    col_anterior = f'rganho_h_{ano - 1:02d}'
    df_trab_filtrada[col_atual] = df_trab_filtrada[col_atual].fillna(df_trab_filtrada[col_anterior])

In [ ]:
df_trab_filtrada[[f'rganho_h_{ano:02d}' for ano in range(10, 20)]].isna().sum()

#### Mudança de Emprego
Criação de colunas que indicam se ocorreu ou não troca de emprego do ano n face ao ano n-1
- Se ambos os valores de nuemp para os dois anos são NaN, retorna False porque não há dados suficientes para determinar uma mudança de emprego.
- Se pelo menos um dos valores de nuemp é NaN, retorna False porque a comparação entre um valor conhecido e um valor ausente não pode determinar com certeza se houve uma mudança de emprego.
- Verifica se os valores de nuemp para os dois anos são diferentes. Se os valores são diferentes, retorna True, indicando que houve uma mudança de emprego entre os dois anos.

In [ ]:
df_emp = df_trab_filtrada.copy()

In [ ]:
def verifica_mudanca_emprego(nuemp_ano1, nuemp_ano2):
    if pd.isna(nuemp_ano1) and pd.isna(nuemp_ano2):
        return False
    if pd.isna(nuemp_ano1) or pd.isna(nuemp_ano2):
        return False
    if nuemp_ano1 != nuemp_ano2:
        return True
    return False

for ano in range(9, 19):
    ano_atual = f'nuemp_{ano:02d}'   
    ano_proximo = f'nuemp_{ano + 1:02d}' 
    nova_coluna = f'mudou_emprego_{ano:02d}_{ano + 1:02d}'  

    df_emp[nova_coluna] = df_emp.apply(lambda row: verifica_mudanca_emprego(row[ano_atual], row[ano_proximo]), axis=1)

df_emp.head()

- Análise da porporção de trocas de emprego

In [ ]:
colunas_mudou_emprego = [col for col in df_emp.columns if col.startswith('mudou_emprego_')]

for coluna in colunas_mudou_emprego:
    contagem = df_emp[coluna].value_counts(dropna=False)
    print(f'Contagem para a coluna {coluna}:')
    print(contagem)
    print()  

## <font color='darkred'>Modelo de Regressão Logistica</font>
### Variáveis Utilizadas no Modelo:
- Mudou de Emprego;
- Habilitações Literárias (habil1);
- Tipo de Contrato;
- Salário (rganho);
- Cae (caem1l);
- Profissão (prof_3d).

In [ ]:
df_regr = df_emp.copy()

In [ ]:
variaveis_interesse = ['habil1', 'tipo_contr1', 'prof_3d', 'rganho_h', 'caem1l']

for ano in range(9, 19):
    for var in variaveis_interesse:
        col_atual = f'{var}_{ano:02d}'
        col_anterior = f'{var}_{ano + 1:02d}'
        nova_coluna = f'diff_{var}_{ano + 1:02d}'
        df_regr[nova_coluna] = df_regr[col_anterior] - df_regr[col_atual]

for ano in range(10, 20):
    print(f"\nAvaliação para a mudança de {ano - 1} para {ano}")
    
    # Definição das variáveis independentes (features) e dependentes (target)
    variaveis_diferenca = [f'diff_{var}_{ano:02d}' for var in variaveis_interesse]
    y = df_regr[f'mudou_emprego_{ano - 1:02d}_{ano:02d}']

    X = df_regr[variaveis_diferenca]
    
    # Normalização de todas as variáveis independentes
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    
    # Conjunto de treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    # Avaliação do modelo
    print("Matriz de Confusão:")
    print(confusion_matrix(y_test, y_pred))
    print("\nRelatório de Classificação:")
    print(classification_report(y_test, y_pred))

    # Importância das features
    importancia_features = pd.Series(model.coef_[0], index=variaveis_diferenca)
    importancia_features = importancia_features.sort_values(ascending=False)
    print("\nImportância das Variáveis:")
    print(importancia_features)

De acordo com os resultados apresentados, é possível verificar que a classe True (mudança de emprego) é significativamente menor em relação à classe False (não mudança de emprego). Isto indica que estamos perante um desequilíbrio de classes. 

### <font color='darkred'> Melhorar o Desempenho do Modelo</font>
#### 1º Método: SMOTE e Class Weights

In [ ]:
variaveis_interesse = ['habil1', 'tipo_contr1', 'prof_3d', 'rganho_h', 'caem1l']

for ano in range(9, 19):
    for var in variaveis_interesse:
        col_atual = f'{var}_{ano:02d}'
        col_anterior = f'{var}_{ano + 1:02d}'
        nova_coluna = f'diff_{var}_{ano + 1:02d}'
        df_regr[nova_coluna] = df_regr[col_anterior] - df_regr[col_atual]

for ano in range(10, 20):
    print(f"\nAvaliação para a mudança de {ano - 1} para {ano}")
    
    # Definição das variáveis independentes (features) e dependentes (target)
    variaveis_diferenca = [f'diff_{var}_{ano:02d}' for var in variaveis_interesse]
    y = df_regr[f'mudou_emprego_{ano - 1:02d}_{ano:02d}']

    X = df_regr[variaveis_diferenca]
    
    # Normalização de todas as variáveis independentes
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    
    # Conjunto de treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # SMOTE para equilibar as classes no conjunto de treino
    smote = SMOTE(random_state=42)
    X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
    
    # Ajustar o modelo de regressão logística com pesos de classe
    model = LogisticRegression(max_iter=1000, class_weight='balanced')
    model.fit(X_train_res, y_train_res)

    y_pred = model.predict(X_test)

    # Avaliar o modelo
    print("Matriz de Confusão:")
    print(confusion_matrix(y_test, y_pred))
    print("\nRelatório de Classificação:")
    print(classification_report(y_test, y_pred))

    # Importância das features
    importancia_features = pd.Series(model.coef_[0], index=variaveis_diferenca)
    importancia_features = importancia_features.sort_values(ascending=False)
    print("\nImportância das Variáveis:")
    print(importancia_features)

#### Conclusões do 1º Método
- **Recall da Classe True:** O recall para a classe True melhorou consideravelmente em comparação com os resultados anteriores. Em vários anos, o recall ficou em torno de 0.50 ou superior, o que significa que o modelo está a considerar uma parte significativa dos casos de mudança de emprego.
- **Precisão da Classe True:** A precisão para a classe True ainda é relativamente baixa, indicando que, embora o modelo esteja a identificar mais mudanças de emprego, ainda há uma quantidade significativa de falsos positivos.

`Importância das Variáveis:`
- **diff_habil1** e **diff_tipo_contr1:** Estas variáveis continuam a ser as mais importantes na maioria das avaliações, o que indica que mudanças na habilitação e no tipo de contrato são fatores significativos na previsão de mudanças de emprego.

### <font color='darkred'> Melhorar o Desempenho do Modelo</font>
#### 2º Método: Ridge Regression

In [ ]:
variaveis_interesse = ['habil1', 'tipo_contr1', 'prof_3d', 'rganho_h', 'caem1l']

for ano in range(9, 19):
    for var in variaveis_interesse:
        col_atual = f'{var}_{ano:02d}'
        col_anterior = f'{var}_{ano + 1:02d}'
        nova_coluna = f'diff_{var}_{ano + 1:02d}'
        df_regr[nova_coluna] = df_regr[col_anterior] - df_regr[col_atual]

for ano in range(10, 20):
    print(f"\nAvaliação para a mudança de {ano - 1} para {ano}")
    
    # Definição das variáveis independentes (features) e dependentes (target)
    variaveis_diferenca = [f'diff_{var}_{ano:02d}' for var in variaveis_interesse]
    y = df_regr[f'mudou_emprego_{ano - 1:02d}_{ano:02d}']

    X = df_regr[variaveis_diferenca]
    
    # Normalizar todas as variáveis independentes
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    
    # Conjunto de treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # SMOTE para equilibar as classes no conjunto de treino
    smote = SMOTE(random_state=42)
    X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
    
    # Configurar o modelo de regressão logística regularizada (Ridge)
    model = LogisticRegression(penalty='l2', max_iter=1000, class_weight='balanced')
    
    # Ajuste de hiperparâmetros usando GridSearchCV
    param_grid = {'C': [0.01, 0.1, 1, 10, 100]}  # Valores de regularização a serem testados
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1', n_jobs=-1)
    grid_search.fit(X_train_res, y_train_res)
    
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)

    # Avaliar o modelo
    print(f"Melhor valor de C (Regularização): {grid_search.best_params_['C']}")
    print("Matriz de Confusão:")
    print(confusion_matrix(y_test, y_pred))
    print("\nRelatório de Classificação:")
    print(classification_report(y_test, y_pred))

    # Importância das features
    importancia_features = pd.Series(best_model.coef_[0], index=variaveis_diferenca)
    importancia_features = importancia_features.sort_values(ascending=False)
    print("\nImportância das Variáveis:")
    print(importancia_features)

## <font color='darkred'>Modelo de Random Forest</font>

- O Random Forest, é um modelo não linear que constrói múltiplas árvores de decisão. Ele capta interações complexas entre variáveis que não seriam capturadas por um modelo linear.
A importância das variáveis no Random Forest é medida com base no nº de vezes que uma variável é usada para dividir os dados nas árvores de decisão, e o impacto dessa divisão na redução da impureza.

In [ ]:
variaveis_interesse = ['habil1', 'tipo_contr1', 'prof_3d', 'rganho_h', 'caem1l']

for ano in range(9, 19):
    for var in variaveis_interesse:
        col_atual = f'{var}_{ano:02d}'
        col_anterior = f'{var}_{ano + 1:02d}'
        nova_coluna = f'diff_{var}_{ano + 1:02d}'
        df_regr[nova_coluna] = df_regr[col_anterior] - df_regr[col_atual]

for ano in range(10, 20):
    print(f"\nAvaliação para a mudança de {ano - 1} para {ano}")
    
    # Definição das variáveis independentes (features) e dependentes (target)
    variaveis_diferenca = [f'diff_{var}_{ano:02d}' for var in variaveis_interesse]
    y = df_regr[f'mudou_emprego_{ano - 1:02d}_{ano:02d}']

    X = df_regr[variaveis_diferenca]
    
    # Normalizar todas as variáveis independentes
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    
    # Conjunto de treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # SMOTE para equilibrar as classes no conjunto de treino
    smote = SMOTE(random_state=42)
    X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
    
    # Modelo RandomForest
    model = RandomForestClassifier(random_state=42, class_weight='balanced', n_jobs=-1)
    
    # Hiperparâmetros com GridSearchCV
    param_grid = {
        'n_estimators': [100, 200, 500],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'bootstrap': [True, False]
    }
    
    # Alternativamente, você pode usar RandomizedSearchCV para maior eficiência
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1', n_jobs=-1)
    grid_search.fit(X_train_res, y_train_res)
    
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)

    # Avaliar o modelo
    print(f"Melhores parâmetros encontrados: {grid_search.best_params_}")
    print("Matriz de Confusão:")
    print(confusion_matrix(y_test, y_pred))
    print("\nRelatório de Classificação:")
    print(classification_report(y_test, y_pred))

    # Importância das features
    importancia_features = pd.Series(best_model.feature_importances_, index=variaveis_diferenca)
    importancia_features = importancia_features.sort_values(ascending=False)
    print("\nImportância das Variáveis:")
    print(importancia_features)

#### Conclusões do Modelo:
`Recall`

Comparado com os modelos anteriores, o Random Forest conseguiu melhorar substancialmente o recall para a classe "True", o que indica que o modelo está identificando melhor as mudanças de emprego.
Por exemplo, o recall varia de 0.64 a 0.76, o que é significativamente melhor do que os modelos de regressão logística, que tinham valores de recall muito baixos.

`Precisão`

A precisão para a classe "True" é menor (variando de 0.23 a 0.47), mas isto é expectável devido ao trade-off entre precisão e recall em problemas de classificação com classes desbalanceadas.

`F1-score`

A f1-score para a classe "True" também aumentou, indicando um melhor equilíbrio entre precisão e recall.

`Importância das Variáveis`

As variáveis mais importantes são **diff_prof_3d** (diferença na profissão), **diff_caem1l** (código de atividade económica), e **diff_rganho** (diferença nos ganhos). Isto significa que mudanças que mudanças na profissão, cae e remuneração são os principais fatores associados à troca de emprego.

## <font color='darkred'>Modelo LightGBM</font>

In [ ]:
warnings.filterwarnings("ignore")
logging.getLogger("lightgbm").setLevel(logging.ERROR)

variaveis_interesse = ['habil1', 'tipo_contr1', 'prof_3d', 'rganho_h', 'caem1l']

for ano in range(9, 19):
    for var in variaveis_interesse:
        col_atual = f'{var}_{ano:02d}'
        col_anterior = f'{var}_{ano + 1:02d}'
        nova_coluna = f'diff_{var}_{ano + 1:02d}'
        df_regr[nova_coluna] = df_regr[col_anterior] - df_regr[col_atual]

for ano in range(10, 20):
    print(f"\nAvaliação para a mudança de {ano - 1} para {ano}")
    
    # Definir variáveis independentes (features) e dependentes (target) para o ano específico
    variaveis_diferenca = [f'diff_{var}_{ano:02d}' for var in variaveis_interesse]
    y = df_regr[f'mudou_emprego_{ano - 1:02d}_{ano:02d}']

    X = df_regr[variaveis_diferenca]
    
    # Normalizar todas as variáveis independentes
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    
    # Conjunto de treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Configurar o modelo LightGBM
    model = lgb.LGBMClassifier(
        objective='binary',
        n_estimators=100,  
        max_depth=5,       
        learning_rate=0.1, 
        subsample=1.0,    
        colsample_bytree=1.0,  
        random_state=42,
        n_jobs=-1,
        verbose=-1  
    )
    
    # Parâmetros para RandomizedSearchCV
    param_dist = {
        'num_leaves': [31, 50, 70],
        'min_child_samples': [20, 50, 100],
        'reg_alpha': [0, 0.1, 0.5],
        'reg_lambda': [0, 0.1, 0.5],
    }
    
    # RandomizedSearchCV para encontrar o melhor modelo
    grid_search = RandomizedSearchCV(
        model, param_distributions=param_dist, n_iter=10, scoring='f1', cv=3, verbose=0, n_jobs=-1, random_state=42
    )
    grid_search.fit(X_train, y_train)
    
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)

    # Avaliar o modelo
    print(f"Melhores parâmetros encontrados: {grid_search.best_params_}")
    print("Matriz de Confusão:")
    print(confusion_matrix(y_test, y_pred))
    print("\nRelatório de Classificação:")
    print(classification_report(y_test, y_pred))

    # Importância das features
    importancia_features = pd.Series(best_model.feature_importances_, index=variaveis_diferenca)
    importancia_features = importancia_features.sort_values(ascending=False)
    print("\nImportância das Variáveis:")
    print(importancia_features)

## <font color='darkred'>Modelo LightGBM (Interações Polinomiais)</font> 

In [ ]:
variaveis_interesse = ['habil1', 'tipo_contr1', 'prof_3d', 'rganho_h', 'caem1l']

for ano in range(9, 19):
    for var in variaveis_interesse:
        col_atual = f'{var}_{ano:02d}'
        col_anterior = f'{var}_{ano + 1:02d}'
        nova_coluna = f'diff_{var}_{ano + 1:02d}'
        df_regr[nova_coluna] = df_regr[col_anterior] - df_regr[col_atual]

for ano in range(10, 20):
    print(f"\nAvaliação para a mudança de {ano - 1} para {ano}")
    
    # Definição das variáveis independentes (features) e dependentes (target)
    variaveis_diferenca = [f'diff_{var}_{ano:02d}' for var in variaveis_interesse]
    y = df_regr[f'mudou_emprego_{ano - 1:02d}_{ano:02d}']
    X = df_regr[variaveis_diferenca]
    
    # Adicionar interações polinomiais
    poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
    X_poly = poly.fit_transform(X)
    
    # Normalizar todas as variáveis independentes
    scaler = StandardScaler()
    X_poly = scaler.fit_transform(X_poly)
    
    # Dividir dados em conjunto de treinamento e teste
    X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.2, random_state=42)
    
    # Configurar o modelo LightGBM
    model = lgb.LGBMClassifier(
        objective='binary',
        n_estimators=100,
        max_depth=5,
        learning_rate=0.1,
        subsample=1.0,
        colsample_bytree=1.0,
        random_state=42,
        n_jobs=-1,
        verbose=-1
    )
    
    # Parâmetros para RandomizedSearchCV
    param_dist = {
        'num_leaves': [31, 50, 70, 150],
        'min_child_samples': [10, 20, 50, 100],
        'reg_alpha': [0, 0.1, 0.5, 1.0],
        'reg_lambda': [0, 0.1, 0.5, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'subsample': [0.6, 0.8, 1.0],
        'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],
    }
    
    # Stratified K-Fold Cross-Validation
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    # Otimização do modelo usando RandomizedSearchCV
    grid_search = RandomizedSearchCV(
        model, param_distributions=param_dist, n_iter=10, scoring='f1', cv=skf, n_jobs=-1, random_state=42
    )
    grid_search.fit(X_train, y_train)
    
    # Melhor modelo encontrado
    best_model = grid_search.best_estimator_

    # Fazer previsões
    y_pred = best_model.predict(X_test)

    # Avaliar o modelo
    print(f"Melhores parâmetros encontrados: {grid_search.best_params_}")
    print("Matriz de Confusão:")
    print(confusion_matrix(y_test, y_pred))
    print("\nRelatório de Classificação:")
    print(classification_report(y_test, y_pred))

    # Importância das features
    importancia_features = pd.Series(best_model.feature_importances_, index=poly.get_feature_names_out(variaveis_diferenca))
    importancia_features = importancia_features.sort_values(ascending=False)
    print("\nImportância das Variáveis:")
    print(importancia_features)


---
Beatriz Lapa - Tese de Mestrado